<a href="https://colab.research.google.com/github/ivanpicavet/automatic__offensive_speech_detection/blob/main/Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle
import numpy as np
import pandas as pd
import re
from nltk.tokenize import word_tokenize
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
punkt=nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
wordnet=nltk.download('wordnet') 

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
av_percept_tagger=nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
stopwordsd=nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
lemmatizer = WordNetLemmatizer() 

In [ ]:
post={"title":"first post","content":"i will give you nice flowers because you are the most beautiful girl in the world"}

In [ ]:
def clean_up(s):
  s=re.sub("@\S+|http\S+|[^a-z\s]"," ",s.lower())
  s = word_tokenize(s)
  s=[word for word in s if not word in stopwords.words()]
  s=" ".join(s)
  return s



In [ ]:
text=clean_up(post["content"])

text

'give nice flowers beautiful girl world'

In [ ]:
with open("bow_vect3","rb") as f:
  bow_vect=pickle.load(f)

In [ ]:
to_pred=bow_vect.transform([text]).toarray()

In [ ]:
with open("new_grad_mod","rb") as f:
  model=pickle.load(f)

In [ ]:
x=model.predict(to_pred)

In [ ]:
x[0]

0

# Nouvelle section

In [ ]:
dftrump=pd.read_csv("./dtrump_results.csv")

In [ ]:
dftrump.head()

,Unnamed: 0,date,language,screen_name,tweet
0,0.0,2020-12-08 05:15:48,en,realDonaldTrump,"Georgia is watching @BrianKempGA, @GeoffDuncan..."
1,1.0,2020-12-08 14:36:34,en,realDonaldTrump,I hope House Republicans will vote against the...
2,2.0,2020-12-07 02:01:28,en,realDonaldTrump,True. Great debate for Kelly! https://t.co/M2y...
3,3.0,2020-12-07 02:37:51,und,realDonaldTrump,https://t.co/RAIj4zXKBk
4,4.0,2020-12-09 14:27:56,en,realDonaldTrump,Germany has consistently been used by my obnox...


In [ ]:
dftrump.drop(["Unnamed: 0","date",],axis=1,inplace=True)

In [ ]:
dftrump.head()

In [ ]:
dftrump=dftrump[dftrump["screen_name"]=="realDonaldTrump"]

In [ ]:
dftrump=dftrump[dftrump["language"]=="en"]

In [ ]:
dftrump.drop(["language","screen_name"],axis=1,inplace=True)

In [ ]:
dftrump.shape

(40228, 1)

In [ ]:
def clean_up(s):
    return re.sub("@\S+|http\S+|[^a-z\s]"," ",s.lower())

In [ ]:
dftrump["clean"]=dftrump["tweet"].apply(clean_up)

In [ ]:
dftrump.head()

,tweet,clean
0,"Georgia is watching @BrianKempGA, @GeoffDuncan...",georgia is watching and
1,I hope House Republicans will vote against the...,i hope house republicans will vote against the...
2,True. Great debate for Kelly! https://t.co/M2y...,true great debate for kelly
4,Germany has consistently been used by my obnox...,germany has consistently been used by my obnox...
5,Thank you to Speaker Cutler and all others in ...,thank you to speaker cutler and all others in ...


In [ ]:
def tokenizer_and_remove_punctuation(row):
    tokens = word_tokenize(row['clean'])
    return [word.lower() for word in tokens if word.isalpha()]



In [ ]:
dftrump['tokenized'] = dftrump.apply(tokenizer_and_remove_punctuation,axis=1)

In [ ]:
wordnet=nltk.download('wordnet') 
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
av_percept_tagger=nltk.download('averaged_perceptron_tagger')
lemmatizer = WordNetLemmatizer() 

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper() # gets first letter of POS categorization
    tag_dict = {"J": wordnet.ADJ, 
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN) 

In [ ]:
def lemmatizer_with_pos(row):   
    return [lemmatizer.lemmatize(word,get_wordnet_pos(word)) for word in row['tokenized']]

In [ ]:
dftrump["lemmatized"]=dftrump.apply(lemmatizer_with_pos,axis=1)

In [ ]:
def remove_sw(row):
    return list(set(row['lemmatized']).difference(stopwords.words()))


In [ ]:

dftrump['no_stopwords'] = dftrump.apply(remove_sw,axis=1)

In [ ]:
def re_blob(row):
    return " ".join(row['no_stopwords'])



In [ ]:
dftrump['clean_blob'] = dftrump.apply(re_blob,axis=1)


In [ ]:
dftrump.head()

,tweet,clean,tokenized,lemmatized,no_stopwords,clean_blob
0,"Georgia is watching @BrianKempGA, @GeoffDuncan...",georgia is watching and,"[georgia, is, watching, and]","[georgia, be, watch, and]","[watch, georgia]",watch georgia
1,I hope House Republicans will vote against the...,i hope house republicans will vote against the...,"[i, hope, house, republicans, will, vote, agai...","[i, hope, house, republican, will, vote, again...","[must, allow, termination, troop, purpose, nat...",must allow termination troop purpose national ...
2,True. Great debate for Kelly! https://t.co/M2y...,true great debate for kelly,"[true, great, debate, for, kelly]","[true, great, debate, for, kelly]","[great, debate, kelly, true]",great debate kelly true
4,Germany has consistently been used by my obnox...,germany has consistently been used by my obnox...,"[germany, has, consistently, been, used, by, m...","[germany, have, consistently, be, use, by, my,...","[obnoxious, follow, love, handle, china, criti...",obnoxious follow love handle china critic argu...
5,Thank you to Speaker Cutler and all others in ...,thank you to speaker cutler and all others in ...,"[thank, you, to, speaker, cutler, and, all, ot...","[thank, you, to, speaker, cutler, and, all, ot...","[corruption, elsewhere, fully, pennsylvania, u...",corruption elsewhere fully pennsylvania unders...


In [ ]:
to_pred_trump=bow_vect.transform(dftrump["clean_blob"]).toarray()

In [ ]:
trump_pred=model.predict(to_pred_trump)

In [ ]:
trump_pred.shape[0]

40228

In [ ]:
prctage_off= sum(trump_pred)/trump_pred.shape[0]

In [ ]:
prctage_off

0.018245997812468925

In [ ]:
kh=pd.read_csv

In [ ]:
kh=pd.read_csv("./kh_results.csv")

In [ ]:
kh.head()

,Unnamed: 0,date,language,screen_name,tweet
0,0.0,2020-12-06 00:49:07,en,KevinHart4real,Thottin 😂😂😂 https://t.co/ZBaqp4lWcY
1,1.0,2020-12-06 20:56:44,en,KevinHart4real,So dope....Much appreciated. Thank you for lov...
2,2.0,2020-11-28 21:21:04,en,KevinHart4real,Just posted a photo https://t.co/ngTqTlgYZS
3,3.0,2020-11-30 21:21:26,en,KevinHart4real,Exciting times 💪🏾💪🏾💪🏾 https://t.co/hUwW3xPCLh
4,4.0,2020-12-09 01:15:14,en,KevinHart4real,Let’s gooooo https://t.co/pcKrB6cuwf


In [ ]:
kh.drop(["Unnamed: 0","date",],axis=1,inplace=True)

In [ ]:
kh.head()

,language,screen_name,tweet
0,en,KevinHart4real,Thottin 😂😂😂 https://t.co/ZBaqp4lWcY
1,en,KevinHart4real,So dope....Much appreciated. Thank you for lov...
2,en,KevinHart4real,Just posted a photo https://t.co/ngTqTlgYZS
3,en,KevinHart4real,Exciting times 💪🏾💪🏾💪🏾 https://t.co/hUwW3xPCLh
4,en,KevinHart4real,Let’s gooooo https://t.co/pcKrB6cuwf


In [ ]:
kh=kh[kh["screen_name"]=="KevinHart4real"]

In [ ]:
kh=kh[kh["language"]=="en"]

In [ ]:
kh.drop(["language","screen_name"],axis=1,inplace=True)

In [ ]:
kh.head()

,tweet
0,Thottin 😂😂😂 https://t.co/ZBaqp4lWcY
1,So dope....Much appreciated. Thank you for lov...
2,Just posted a photo https://t.co/ngTqTlgYZS
3,Exciting times 💪🏾💪🏾💪🏾 https://t.co/hUwW3xPCLh
4,Let’s gooooo https://t.co/pcKrB6cuwf


In [ ]:
def clean_up(s):
    return re.sub("@\S+|http\S+|[^a-z\s]"," ",s.lower())

In [ ]:
kh["clean"]=kh["tweet"].apply(clean_up)

In [ ]:
def tokenizer_and_remove_punctuation(row):
    tokens = word_tokenize(row['clean'])
    return [word.lower() for word in tokens if word.isalpha()]



In [ ]:
kh['tokenized'] = kh.apply(tokenizer_and_remove_punctuation,axis=1)

In [ ]:
wordnet=nltk.download('wordnet') 
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
av_percept_tagger=nltk.download('averaged_perceptron_tagger')
lemmatizer = WordNetLemmatizer() 

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper() # gets first letter of POS categorization
    tag_dict = {"J": wordnet.ADJ, 
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN) 

In [ ]:
def lemmatizer_with_pos(row):   
    return [lemmatizer.lemmatize(word,get_wordnet_pos(word)) for word in row['tokenized']]

In [ ]:
kh["lemmatized"]=kh.apply(lemmatizer_with_pos,axis=1)

In [ ]:
def remove_sw(row):
    return list(set(row['lemmatized']).difference(stopwords.words()))


In [ ]:

kh['no_stopwords'] = kh.apply(remove_sw,axis=1)

In [ ]:
def re_blob(row):
    return " ".join(row['no_stopwords'])



In [ ]:
kh['clean_blob'] = kh.apply(re_blob,axis=1)


In [ ]:
kh.head()

,tweet,clean,tokenized,lemmatized,no_stopwords,clean_blob
0,Thottin 😂😂😂 https://t.co/ZBaqp4lWcY,thottin,[thottin],[thottin],[thottin],thottin
1,So dope....Much appreciated. Thank you for lov...,so dope much appreciated thank you for lov...,"[so, dope, much, appreciated, thank, you, for,...","[so, dope, much, appreciate, thank, you, for, ...","[happen, guy, year, much, fan, continue, thank...",happen guy year much fan continue thank well l...
2,Just posted a photo https://t.co/ngTqTlgYZS,just posted a photo,"[just, posted, a, photo]","[just, post, a, photo]","[photo, post]",photo post
3,Exciting times 💪🏾💪🏾💪🏾 https://t.co/hUwW3xPCLh,exciting times,"[exciting, times]","[excite, time]","[time, excite]",time excite
4,Let’s gooooo https://t.co/pcKrB6cuwf,let s gooooo,"[let, s, gooooo]","[let, s, gooooo]","[let, gooooo]",let gooooo


In [ ]:
to_pred_kh=bow_vect.transform(kh["clean_blob"]).toarray()

In [ ]:
kh_pred=model.predict(to_pred_kh)

In [ ]:
kh_pred.shape[0]

25838

In [ ]:
prctage_off= sum(kh_pred)/kh_pred.shape[0]

In [ ]:
prctage_off

0.16406068581159533